
# HISOL Multi-Scenario Analysis Notebook

Root directory:
**/home/jeanhuit/Documents/Thesis/results**

Auto-discovers scenarios:
- highway
- mixed
- urban

Vehicle densities:
- 50, 100, 150

Each run must contain:
- bsm_log.csv  
- rssi_log.csv  
- neighbor_log.csv  
- sybil_log.csv  
- replay_log.csv  
- jammer_log.csv  


In [ ]:

import os, pandas as pd

ROOT = "/home/jeanhuit/Documents/Thesis/results"

scenario_dirs = ['highway','mixed','urban']
densities = ['density-50','density-100','density-150']

runs = []

for scen in scenario_dirs:
    for dens in densities:
        runpath = os.path.join(ROOT, scen, dens, "run-1")
        if os.path.exists(runpath):
            files = {
                f: os.path.join(runpath, f)
                for f in [
                    'bsm_log.csv','rssi_log.csv','neighbor_log.csv',
                    'sybil_log.csv','replay_log.csv','jammer_log.csv'
                ] if os.path.exists(os.path.join(runpath,f))
            }
            runs.append({'scenario': scen, 'density': dens, 'path': runpath, 'files': files})

runs


In [ ]:

import pandas as pd

loaded = []

for r in runs:
    data = {}
    for name, path in r['files'].items():
        try:
            data[name] = pd.read_csv(path)
        except:
            data[name] = None
    r['data'] = data
    loaded.append(r)

len(loaded)


In [ ]:

import numpy as np
import pandas as pd

results = []

for r in loaded:
    bsm = r['data'].get('bsm_log.csv')
    rssi = r['data'].get('rssi_log.csv')
    neighbor = r['data'].get('neighbor_log.csv')

    sent = len(bsm) if bsm is not None else np.nan
    recv = len(rssi) if rssi is not None else np.nan
    pdr = recv/sent if sent and recv else np.nan
    avg_neighbors = neighbor['count'].mean() if neighbor is not None else np.nan

    results.append({
        "scenario": r["scenario"],
        "density": r["density"],
        "sent": sent,
        "recv": recv,
        "pdr": pdr,
        "avg_neighbors": avg_neighbors
    })

df_results = pd.DataFrame(results)
df_results
